In [105]:
import dicom as dcm
import SimpleITK as sitk
import numpy as np 
import array
from numpy import unravel_index
import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import print_function
from ipywidgets import interact

In [106]:
def ct_viewer(im):
    def plot_image(myslice):
        plt.imshow(im[myslice], cmap=plt.cm.jet)
    interact(plot_image, myslice=(0, im.shape[0]))
    return;

In [107]:
#Choose Patient
study='VILLENEUVE'

In [86]:
rp=dcm.read_file(study+'/dicom/')
#rd=dcm.read_file(study+'/dicom/rd_joue.dcm')
print(rp)
#get pixel

IOError: [Errno 21] Is a directory: 'VILLENEUVE/dicom/'

### Edit rt dose.dcm in raw file

In [108]:
rp=dcm.read_file(study+'/dicom/RP.201700810.sternum.dcm')
rd=dcm.read_file(study+'/dicom/RD.201700810.Dose_sternum.dcm')

#get pixel array
rdpix=rd.pixel_array.astype(np.float32)


#Convert rt dose value into Gy value 
f=float(rd.DoseGridScaling)
print(f)
rdpix1=rdpix*f 

#create an sitk image from this array
rdim=sitk.GetImageFromArray(rdpix1)

#Set origin of image from rt dose value
origin=[]
for val in rd.ImagePositionPatient: origin.append(float(val))
rdim.SetOrigin(origin)
#Set spacing of image from rt dose value (x et y)
spacing=[]
for val in rd.PixelSpacing: spacing.append(float(val))
#for z direction
spacez=float(rd.GridFrameOffsetVector[1]-rd.GridFrameOffsetVector[0])
spacing.append(spacez)
rdim.SetSpacing(spacing)

#write rd image in '.raw'
sitk.WriteImage(rdim, study+'/results/eclipse_'+study+'.mhd')
print('eclipse_'+study+'.mhd --> ok')

3.0624745e-05
eclipse_VILLENEUVE.mhd --> ok


In [109]:
#Get the rd raw created  and gate image
refim=sitk.ReadImage(study+'/results/eclipse_'+study+'.mhd')
gateim=sitk.ReadImage(study+'/results/villeneuve_out.mhd')

In [110]:
# get pixels arrays of images and vizualization
gatepix=sitk.GetArrayFromImage(gateim)
refpix=sitk.GetArrayFromImage(refim)                                          
ct_viewer(refpix)
ct_viewer(gatepix)

interactive(children=(IntSlider(value=104, description=u'myslice', max=208), Output()), _dom_classes=('widget-interact',))

interactive(children=(IntSlider(value=104, description=u'myslice', max=208), Output()), _dom_classes=('widget-interact',))

In [111]:
#to mask GATE images an get just BODY DOSE
#Resample (Downsampling) rd image into gate image size an resolution
refim_res=sitk.Resample(refim, gateim, sitk.Transform(), sitk.sitkNearestNeighbor, 0)
#Get pixel array
refim_respix=sitk.GetArrayFromImage(refim_res)
ct_viewer(refim_respix)

interactive(children=(IntSlider(value=104, description=u'myslice', max=208), Output()), _dom_classes=('widget-interact',))

In [112]:
#Write resampled image
sitk.WriteImage(refim_res, study+'/results/eclipse_'+study+'_resampled.mhd')
print('eclipse_'+study+'_resampled.mhd --> ok')

eclipse_VILLENEUVE_resampled.mhd --> ok


In [113]:
#Create a boolean mask to apply on Gate image (remove dose outside the patient according to RTdose)
mask=np.ma.make_mask(refim_respix)
#apply on gate pixel array
gatepix_mask=gatepix*mask
ct_viewer(gatepix_mask)

interactive(children=(IntSlider(value=104, description=u'myslice', max=208), Output()), _dom_classes=('widget-interact',))

In [114]:
#Absolute dose conversion LUC SIMON
um=float(rp.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
nFrac=float(rp.FractionGroupSequence[0].NumberofFractionsPlanned)

GVref6=0.000108726
GVref9=0.000140945
GVref12=0.000150559

gatepix_abs=gatepix_mask*(1.0/GVref12)*0.01*um*nFrac
print(gatepix_abs.max())

32.0589


In [81]:
#Absolute dose conversion BIBLIO

Npart_6=1033128667.0
Npart_9=1136273663.0
Npart_12=1086793583.0
DrG_6=0.000108726
DrG_9=0.000140945
DrG_12=0.000150559

GperPart6=DrG_6/Npart_6
GperPart9=DrG_9/Npart_9
GperPart12=DrG_12/Npart_12
Dref=0.01
F6=Dref/GperPart6
F9=Dref/GperPart9
F12=Dref/GperPart12

um=float(rp.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
nFrac=float(rp.FractionGroupSequence[0].NumberofFractionsPlanned)

gatepix_abs=(gatepix_mask/Npart_6)*F6*um*nFrac
print(gatepix_mask.max())
print(gatepix_abs.max())


0.000116792
59.2951


In [115]:

#Create a raw image of gate with mask
#gate_dose=sitk.GetImageFromArray(gatepix_mask)
gate_dose=sitk.GetImageFromArray(gatepix_abs)
gate_dose.SetOrigin(gateim.GetOrigin())
gate_dose.SetSpacing(gateim.GetSpacing())
sitk.WriteImage(gate_dose, study+'/results/gate_'+study+'.mhd')